In [1]:
import cv2 as cv
import os
import time

In [2]:
video = cv.VideoCapture('my_video.mp4')

In [3]:
# Display relative video propreties
print("Video frame width: \t  ", video.get(cv.CAP_PROP_FRAME_WIDTH))
print("Video frame height: \t  ", video.get(cv.CAP_PROP_FRAME_HEIGHT))
print("Video frame rate property:", video.get(cv.CAP_PROP_FPS))

Video frame width: 	   640.0
Video frame height: 	   360.0
Video frame rate property: 25.0


In [4]:
frame_nb = 0
frames_left = True

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    frame_name = os.path.join('frames', f"frame_{frame_nb:04d}.jpg")
    cv.imwrite(frame_name, frame) # saves the frame
    cv.imshow('Frame', frame)     # Show frame 
    frame_nb += 1
    key = cv.waitKey(50)

print(f"{frame_nb} frames saved.")
video.release()
cv.destroyAllWindows()


160 frames saved.


In [5]:
# Spatial Segmentation:
video = cv.VideoCapture('my_video.mp4')
frame_nb = 0
frames_left = True

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    cv.imshow('Frame', cv.cvtColor(frame, cv.COLOR_BGR2GRAY)) 
    frame_nb += 1
    key = cv.waitKey(50)

video.release()
cv.destroyAllWindows()


In [6]:
frame = cv.imread('frames/frame_0089.jpg')
gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
cv.imshow('Gray Frame', gray_frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [11]:
# Algo de seuillage global heuristique
T = 25
for i in range(5):
    print(f"Iteration {i+1}, T = {T}")
    _, binary_frame = cv.threshold(gray_frame, T, 255, cv.THRESH_BINARY)
    mean1 = gray_frame[binary_frame == 255].mean()
    mean2 = gray_frame[binary_frame == 0].mean()
    T = (mean1 + mean2) / 2
    cv.imshow('Binary Frame', binary_frame)
    cv.waitKey(0)
cv.destroyAllWindows()

Iteration 1, T = 25
Iteration 2, T = 82.70778794334248
Iteration 3, T = 94.15467179766662
Iteration 4, T = 94.4257879577422
Iteration 5, T = 94.4257879577422
